# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e compramos e vendemos commodities:
- Soja, Milho, Trigo, Petróleo, etc.

Precisamos pegar na internet, de forma automática, a cotação de todas as commodites e ver se ela está abaixo do nosso preço ideal de compra. Se tiver, precisamos marcar como uma ação de compra para a equipe de operações.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=share_link

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [6]:
# Recomendacao do selenium utilizar os navegadores Firefox ou Google Chrome

# webdriver
# firefox -> geckodriver
# chrome -> chromedriver

from selenium import webdriver # importa o modulo de utilizacao do browser do selenium
import time


# Passo a passo

## Passo 1: Abrir o navegador

navegador=webdriver.Chrome()
navegador.get('https://www.google.com/') # no jupyter o navegador nao se fecha ao final

time.sleep(1)
# navegador=webdriver.Firefox()


# Passo 2: Importar a base de dados

import pandas as pd

tabela=pd.read_excel('commodities.xlsx')
display(tabela)


# Passo 3: Para cada produto da base de dados
# Passo 4: Pesquisar o preco do produto
# Passo 5: atualizar o preço na base de dados

"""
Elemento em questão abaixo:

<input type="text" value="85,49" class="text-verde" id="comercial" calc="sim">
"""

# Opções para serem feitas com os elementos:

# .click() -> clicar
# .send_keys('texto') -> escrever
# .get_attribute() - > pegar um valor  

for linha in tabela.index:
    produto=tabela.loc[linha,'Produto']
    print(produto)
    produto=produto.replace('ó','o').replace('ã','a').replace('á','a').replace('ç','c').replace('é','e').replace('ú','u').lower() # retirando todos os caracteres especiais para que o link nao quebre na consulta

    link=f'https://www.melhorcambio.com/{produto}-hoje'
    navegador.get(link)

    precoProduto=navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value') # usar copy xpath do site no elemento desejado (CTRL+SHIFT+C -> CLICAR NO ELEMENTO DESEJADO -> CLICAR SEGUNDO BOTAO MOUSE -> COPY XPATH)
    print(precoProduto)
    tabela.loc[linha,'Preço Atual']=float(precoProduto.replace('.','').replace(',','.'))

    # preencher a coluna de comprar
    tabela.loc[linha,'Comprar']=True if tabela.loc[linha,'Preço Atual']<tabela.loc[linha,'Preço Ideal'] else False

# tabela['Comprar'] = tabela['Preço Atual'] < tabela['Preço Ideal'] # opcao fora do for que TAMBEM preenche automaticamente a coluna comprar sem precisar de um laco de repeticao pra isso

display(tabela)


# exportar a base para o excel

tabela.to_excel('commodities_atualizado.xlsx', index=False) # usamos o index=False para que ele nao exporte tambem a coluna de indices

navegador.quit() # no jupyter o navegador nao fecha automaticamente, entao faremos o fechamento manual


# Passo 6: Decidir quais produtos comprar

,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,NaN,NaN
1,Soja,163.59,NaN,NaN
2,Boi,282.20,NaN,NaN
3,Petróleo,424.37,NaN,NaN
4,Algodão,497.76,NaN,NaN
5,Açúcar,136.23,NaN,NaN
6,Café,1092.87,NaN,NaN
7,Ouro,321.77,NaN,NaN
8,Trigo,1549.11,NaN,NaN
9,Tilápia,9.05,NaN,NaN


Milho
0,00
Soja
0,00
Boi
0,00
Petróleo
392,97
Algodão
0,00
Açúcar
0,00
Café
0,00
Ouro
326,57
Trigo
0,00
Tilápia
0,00


,Produto,Preço Ideal,Preço Atual,Comprar
0,Milho,85.32,0.00,True
1,Soja,163.59,0.00,True
2,Boi,282.20,0.00,True
3,Petróleo,424.37,392.97,True
4,Algodão,497.76,0.00,True
5,Açúcar,136.23,0.00,True
6,Café,1092.87,0.00,True
7,Ouro,321.77,326.57,False
8,Trigo,1549.11,0.00,True
9,Tilápia,9.05,0.00,True
